In [1]:
import os
from PIL import Image
import numpy as np
from tifffile import imread

In [2]:
def get_coordinates_and_ids_from_tif(tif_file):
    image = imread(tif_file)
    if len(image.shape) == 2:
        height, width = image.shape
    elif len(image.shape) == 3:
        height, width, _ = image.shape
    else:
        raise ValueError(f"Unexpected image shape {image.shape}")

    unique_ids = np.unique(image)
    unique_ids = unique_ids[unique_ids != 0]

    coords_and_ids = []
    for cell_id in unique_ids:
        y, x = np.where(image == cell_id)
        
        # Compute the mean of x and y and normalize
        mean_x = np.mean(x) / width
        mean_y = np.mean(y) / height
        
        coords_and_ids.append((mean_x, mean_y, cell_id))
    
    return coords_and_ids

def process_folder(directory):
    for folder in os.listdir(directory):
        if folder.endswith('_GT'):
            tra_folder = os.path.join(directory, folder, 'TRA')
            
            # Load track info
            with open(os.path.join(tra_folder, 'man_track.txt'), 'r') as f:
                lines = f.readlines()

            track_info = {}
            for line in lines:
                L, B, E, P = map(int, line.strip().split())
                track_info[L] = (B, E, P)

            output_data = []
            tif_files = sorted([f for f in os.listdir(tra_folder) if f.startswith('man_track') and f.endswith('.tif')])
            
            for tif_file in tif_files:
                frame_number = int(tif_file[len("man_track"):-len(".tif")])
                coords_and_ids = get_coordinates_and_ids_from_tif(os.path.join(tra_folder, tif_file))
                
                for x, y, cell_id in coords_and_ids:
                    B, E, P = track_info[cell_id]
                    if B <= frame_number <= E:
                        # Set split_id only for the frame where the split happens
                        split_id = P if (P and B == frame_number) else 0
                        output_data.append((x, y, 1, cell_id, split_id, 1, frame_number))
            
            with open(os.path.join(tra_folder, 'pos_GT.txt'), 'w') as f:
                f.write("# x\ty\tr\tid\tsplit_id\ts_prob\tt\n")
                for x, y, r, cell_id, split_id, s_prob, t in output_data:
                    f.write(f"{x}\t{y}\t{r}\t{cell_id}\t{split_id}\t{s_prob}\t{t}\n")





In [3]:
directory = '/media/mo/Label/HUH_01_t/'
process_folder(directory)